In [1]:
import cvxpy as cp
import numpy as np

from basis_generator import basis_gs, sel_seq

# Guess Your Neighboors Input parameters

In [2]:
dim = 2
num_obs = 2
len_seq = 3
out_max = 1
basis_size= 15
seq_method="sel_sequences"

GYN_events = [((0, 0, 0), (0, 0, 0)),
            ((1, 1, 0), (0, 1, 1)),
            ((0, 1, 1), (1, 0, 1)),
            ((1, 0, 1), (1, 1, 0)),]

# Optimization with the First Level of the Hierarchy

In [3]:
X_basis = basis_gs(dim = 2,
            num_obs = 3,
            len_seq = 2,
            num_out = 2,
            prec = 1e-15,
            stop = 10000)

Nul matrix found
Number of LI moment matrices:  52


In [4]:
alpha = cp.Variable((len(X_basis), 1))
X = cp.Variable(X_basis[0].shape, symmetric=True)

constraints = [X >> 0]
constraints += [
    X == sum([alpha[i]*X_basis[i] for i in range(len(X_basis))])
]
constraints += [
    cp.trace(X) == num_obs**len_seq
]

In [5]:
sequences = sel_seq(num_obs = 3,
                len_seq = 2,
                num_out = 2,
                remove_last_out = False)

C = np.zeros(X_basis[0].shape)
for i, seq_row in enumerate(sequences):
    if seq_row in GYN_events:
        C[i,i] = 1
        
prob = cp.Problem(cp.Maximize(cp.trace(C@X)),
                  constraints)

In [6]:
prob.solve(solver=cp.MOSEK, verbose=False)

print(prob.status)
print("The optimal value is", prob.value)

optimal
The optimal value is -0.0
